In [1]:
!pip install wandb

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import random_split
import wandb

In [3]:
train_dir = '/kaggle/input/inaturalist-data-set/inaturalist_12K/train'
test_dir = '/kaggle/input/inaturalist-data-set/inaturalist_12K/val'

#train_dir = '/kaggle/input/inaturalist/inaturalist_12K/train'
#test_dir = '/kaggle/input/inaturalist/inaturalist_12K/val'
IMG_SIZE = (224,224)

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize the images
    transforms.ToTensor(),        # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])  # Normalize the images


# Use ImageFolder to create a dataset
trainset = datasets.ImageFolder(root=train_dir, transform=transform)
val_size = int(0.2 * len(trainset))
train_size = len(trainset) - val_size
trainset, valset = random_split(trainset, [train_size, val_size])

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
class pretrainedCNN():
  def __init__(self, trainset, valset, model, batch_size=32, freeze_percent = 1):
    # freeze earlier layers
    self.model = model
    if freeze_percent==1:
      for param in self.model.parameters():
        param.requires_grad = False
    else:
      total_layers = sum(1 for _ in model.parameters())
      num_layers_to_freeze = int(freeze_percent * total_layers)

      # Counter to track the number of frozen layers
      frozen_layers = 0

      # Freeze layers until reaching the desired percentage
      for param in self.model.parameters():
          param.requires_grad = False
          frozen_layers += 1
          if frozen_layers >= num_layers_to_freeze:
              break

    # Modify fully connected layer for fine-tuning
    num_classes = 10  # Number of classes in iNaturalist dataset
    num_ftrs = self.model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, num_classes)
    for param in self.model.fc.parameters():
      param.requires_grad = True
    # DataLoader for the dataset
    self.dataloader_train = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True)
    self.dataloader_val = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle=True)

  def train(self, epochs = 10, lr=0.001, weight_decay=0):
    self.model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(1,epochs+1):
      train_accuracy = 0
      count = 0
      self.model.train()
      for inputs, labels in self.dataloader_train:
          # forward, backward, and then weight update
          inputs = inputs.to(device)
          labels = labels.to(device)
          # Forward pass
          optimizer.zero_grad()
          y_pred = self.model(inputs)
          loss = criterion(y_pred, labels)
          train_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
          count += len(labels)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      train_accuracy /= count
      print("Epoch %d: model train accuracy %.2f%%" % (epoch, train_accuracy*100))
      wandb.log({ 'epoch': epoch, 'train_accuracy': train_accuracy * 100})
      self.model.eval()
      with torch.no_grad():
        val_accuracy = 0
        count = 0
        for inputs, labels in self.dataloader_val:
            inputs = inputs.to(device)
            labels = labels.to(device)
            y_pred = self.model(inputs)
            val_loss = criterion(y_pred, labels)
            val_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
            count += len(labels)
        val_accuracy /= count
        print("Epoch %d: model validation accuracy %.2f%%" % (epoch, val_accuracy*100))
        wandb.log({ 'epoch': epoch, 'validation_accuracy': val_accuracy * 100})
        wandb.log({ 'epoch': epoch, 'validation_loss': val_loss * 100})

In [5]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

key = input('Enter your API:')
wandb.login(key=key) #key="7836c2516a56fb9071600184ea90a0f6357f99dc"

Enter your API: 7836c2516a56fb9071600184ea90a0f6357f99dc


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tentuvenkatesh2 (tentuvenkatesh2-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
# sweep config file
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep - pretuning',
    'metric': {
      'goal': 'maximize',
      'name': 'validation_accuracy'
    },
    'parameters': {
        'lr': {
            'values': [1e-3, 1e-4]
        },
        'freeze_percent':{
            'values': [0.2, 0.6, 0.9]
        },
        'l2_reg':{
            'values': [0, 0.0005, 0.05]
        },
        'batch_size': {
            'values': [32, 64]
        },
        'epochs':{
            'values': [5, 10]   
        }
    }
}

In [10]:
# Create a sweep
sweep_id = wandb.sweep(sweep = sweep_config,project='DA6401-AS2_P2_10')

Create sweep with ID: qyxi9lr0
Sweep URL: https://wandb.ai/tentuvenkatesh2-indian-institute-of-technology-madras/DA6401-AS2_P2_10/sweeps/qyxi9lr0


In [8]:

def main():
  with wandb.init() as run:
    run_name = "-bs_"+str(wandb.config.batch_size) +"-ep_"+str(wandb.config.epochs)+"-lr_"+str(wandb.config.lr) \
                  +"-freeze_"+str(wandb.config.freeze_percent)
    wandb.run.name = run_name
    from torchvision.models import ResNet50_Weights
    pre_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    cnn_model = pretrainedCNN(trainset, valset, pre_model, batch_size=wandb.config.batch_size, freeze_percent = wandb.config.freeze_percent)
    cnn_model.train(wandb.config.epochs, wandb.config.lr, wandb.config.l2_reg)
wandb.agent(sweep_id, function = main, count = 1)
wandb.finish()

wandb: Agent Starting Run: jwpeqd11 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	freeze_percent: 0.2
wandb: 	l2_reg: 0.0005
wandb: 	lr: 0.0001
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 188MB/s] 


Epoch 1: model train accuracy 69.86%
Epoch 1: model validation accuracy 77.19%
Epoch 2: model train accuracy 90.49%
Epoch 2: model validation accuracy 75.74%
Epoch 3: model train accuracy 96.61%
Epoch 3: model validation accuracy 75.74%
Epoch 4: model train accuracy 98.31%
Epoch 4: model validation accuracy 77.14%
Epoch 5: model train accuracy 97.99%
Epoch 5: model validation accuracy 76.14%
Epoch 6: model train accuracy 98.19%
Epoch 6: model validation accuracy 74.64%
Epoch 7: model train accuracy 98.63%
Epoch 7: model validation accuracy 76.14%
Epoch 8: model train accuracy 98.19%
Epoch 8: model validation accuracy 74.39%
Epoch 9: model train accuracy 97.90%
Epoch 9: model validation accuracy 75.19%
Epoch 10: model train accuracy 98.08%
Epoch 10: model validation accuracy 74.69%


epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
train_accuracy,▁▆████████
validation_accuracy,█▄▄█▅▂▅▁▃▂
validation_loss,▃▁▂▁▄▃▂▃▃█
epoch,10
train_accuracy,98.075
validation_accuracy,74.68735
validation_loss,209.42751


In [11]:

def main():
  with wandb.init() as run:
    run_name = "-bs_"+str(wandb.config.batch_size) +"-ep_"+str(wandb.config.epochs)+"-lr_"+str(wandb.config.lr) \
                  +"-freeze_"+str(wandb.config.freeze_percent)
    wandb.run.name = run_name
    from torchvision.models import ResNet50_Weights
    pre_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    cnn_model = pretrainedCNN(trainset, valset, pre_model, batch_size=wandb.config.batch_size, freeze_percent = wandb.config.freeze_percent)
    cnn_model.train(wandb.config.epochs, wandb.config.lr, wandb.config.l2_reg)
wandb.agent(sweep_id, function = main, count = 10)
wandb.finish()

wandb: Agent Starting Run: 86p16e2g with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	freeze_percent: 0.9
wandb: 	l2_reg: 0
wandb: 	lr: 0.0001


Epoch 1: model train accuracy 70.36%
Epoch 1: model validation accuracy 76.69%
Epoch 2: model train accuracy 83.23%
Epoch 2: model validation accuracy 76.74%
Epoch 3: model train accuracy 90.75%
Epoch 3: model validation accuracy 77.99%
Epoch 4: model train accuracy 95.91%
Epoch 4: model validation accuracy 77.54%
Epoch 5: model train accuracy 97.86%
Epoch 5: model validation accuracy 77.14%
Epoch 6: model train accuracy 98.29%
Epoch 6: model validation accuracy 76.69%
Epoch 7: model train accuracy 98.75%
Epoch 7: model validation accuracy 77.94%
Epoch 8: model train accuracy 98.81%
Epoch 8: model validation accuracy 77.79%
Epoch 9: model train accuracy 99.24%
Epoch 9: model validation accuracy 78.59%
Epoch 10: model train accuracy 99.03%
Epoch 10: model validation accuracy 77.64%


epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
train_accuracy,▁▄▆▇██████
validation_accuracy,▁▁▆▄▃▁▆▅█▅
validation_loss,▅▄▃▁█▅▅▄▂▃
epoch,10
train_accuracy,99.025
validation_accuracy,77.63882
validation_loss,66.15424


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: l33e5ul4 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	freeze_percent: 0.9
wandb: 	l2_reg: 0.05
wandb: 	lr: 0.001


Epoch 1: model train accuracy 62.31%
Epoch 1: model validation accuracy 60.98%
Epoch 2: model train accuracy 65.79%
Epoch 2: model validation accuracy 61.43%
Epoch 3: model train accuracy 68.34%
Epoch 3: model validation accuracy 68.58%
Epoch 4: model train accuracy 70.60%
Epoch 4: model validation accuracy 64.18%
Epoch 5: model train accuracy 72.56%
Epoch 5: model validation accuracy 70.79%
Epoch 6: model train accuracy 74.39%
Epoch 6: model validation accuracy 73.44%
Epoch 7: model train accuracy 74.14%
Epoch 7: model validation accuracy 69.93%
Epoch 8: model train accuracy 75.04%
Epoch 8: model validation accuracy 71.69%
Epoch 9: model train accuracy 75.01%
Epoch 9: model validation accuracy 71.94%
Epoch 10: model train accuracy 75.06%
Epoch 10: model validation accuracy 71.24%


...

Epoch 1: model train accuracy 70.15%
Epoch 1: model validation accuracy 77.39%
Epoch 2: model train accuracy 83.65%
Epoch 2: model validation accuracy 78.24%
Epoch 3: model train accuracy 90.65%
Epoch 3: model validation accuracy 77.14%
Epoch 4: model train accuracy 95.49%
Epoch 4: model validation accuracy 77.49%
Epoch 5: model train accuracy 97.58%
Epoch 5: model validation accuracy 77.94%
Epoch 6: model train accuracy 98.30%
Epoch 6: model validation accuracy 77.84%
Epoch 7: model train accuracy 99.03%
Epoch 7: model validation accuracy 77.14%
Epoch 8: model train accuracy 98.80%
Epoch 8: model validation accuracy 78.29%
Epoch 9: model train accuracy 98.34%
Epoch 9: model validation accuracy 77.29%
Epoch 10: model train accuracy 98.56%
Epoch 10: model validation accuracy 77.74%


epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
train_accuracy,▁▄▆▇██████
validation_accuracy,▃█▁▃▆▅▁█▂▅
validation_loss,▃▁▂▅▁█▃▁▂▆
epoch,10
train_accuracy,98.5625
validation_accuracy,77.73887
validation_loss,118.12886


wandb: Agent Starting Run: 7mg1ov6h with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	freeze_percent: 0.9
wandb: 	l2_reg: 0
wandb: 	lr: 0.0001


Epoch 1: model train accuracy 70.29%
Epoch 1: model validation accuracy 76.64%
Epoch 2: model train accuracy 83.43%
Epoch 2: model validation accuracy 78.09%
Epoch 3: model train accuracy 91.03%
Epoch 3: model validation accuracy 78.44%
Epoch 4: model train accuracy 95.56%
Epoch 4: model validation accuracy 78.64%
Epoch 5: model train accuracy 97.68%
Epoch 5: model validation accuracy 77.19%
Epoch 6: model train accuracy 98.13%
Epoch 6: model validation accuracy 77.24%
Epoch 7: model train accuracy 98.83%
Epoch 7: model validation accuracy 78.04%
Epoch 8: model train accuracy 98.86%
Epoch 8: model validation accuracy 75.79%
Epoch 9: model train accuracy 98.61%
Epoch 9: model validation accuracy 76.79%
Epoch 10: model train accuracy 98.95%
Epoch 10: model validation accuracy 77.84%


epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
train_accuracy,▁▄▆▇██████
validation_accuracy,▃▇██▄▅▇▁▃▆
validation_loss,▂▁█▃▃▆▃▅▂▃
epoch,10
train_accuracy,98.95
validation_accuracy,77.83892
validation_loss,65.90424


**Based on 10 epochs best one i got is train_accuracy-99.16251 and validation_accuracy-77.83892**